In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R
import random
from Bio import pairwise2
from Bio.Align import PairwiseAligner  # Modern API - more accurate
from Bio.Seq import Seq
import time
# Removed distance_matrix import - using pairwise calculation instead for memory efficiency
import warnings

warnings.filterwarnings('ignore')

# === 1. Loading Data ===
DATA_PATH = '/kaggle/input/stanford-rna-3d-folding-2/'

train_seqs = pd.read_csv(DATA_PATH + 'train_sequences.csv')
test_seqs = pd.read_csv(DATA_PATH + 'test_sequences.csv')
train_labels = pd.read_csv(DATA_PATH + 'train_labels.csv')

try:
    validation_seqs = pd.read_csv(DATA_PATH + 'validation_sequences.csv')
    validation_labels = pd.read_csv(DATA_PATH + 'validation_labels.csv')
    print("Validation data found and will be combined with train data.")
    
    combined_seqs = pd.concat([train_seqs, validation_seqs], ignore_index=True)
    
    combined_labels = pd.concat([train_labels, validation_labels], ignore_index=True)
    
except FileNotFoundError:
    print("Validation data not found, using only train data.")
    combined_seqs = train_seqs
    combined_labels = train_labels

def process_labels(labels_df):
    coords_dict = {}
    for id_prefix, group in labels_df.groupby(lambda x: labels_df['ID'][x].rsplit('_', 1)[0]):
        coords = [group.sort_values('resid')[['x_1', 'y_1', 'z_1']].values]
        coords_dict[id_prefix] = coords[0]
    return coords_dict

combined_coords_dict = process_labels(combined_labels)

# Initialize modern aligner (more accurate than pairwise2)
MODERN_ALIGNER = PairwiseAligner()
MODERN_ALIGNER.mode = 'global'
MODERN_ALIGNER.match_score = 2.0
MODERN_ALIGNER.mismatch_score = -1.0
MODERN_ALIGNER.open_gap_score = -3.0
MODERN_ALIGNER.extend_gap_score = -0.1

# === 2nd Phase - IMPROVED ===

def fast_sequence_similarity(seq1, seq2):
    """
    FAST APPROXIMATION: Use k-mer counting instead of full alignment
    This is 100-1000x faster than pairwise alignment for initial filtering
    """
    k = 3  # Use 3-mers (trigrams)
    if len(seq1) < k or len(seq2) < k:
        return 0.0
    
    # Count k-mers
    def get_kmers(seq, k):
        return set(seq[i:i+k] for i in range(len(seq) - k + 1))
    
    kmers1 = get_kmers(seq1, k)
    kmers2 = get_kmers(seq2, k)
    
    if not kmers1 or not kmers2:
        return 0.0
    
    # Jaccard similarity of k-mer sets
    intersection = len(kmers1 & kmers2)
    union = len(kmers1 | kmers2)
    return intersection / union if union > 0 else 0.0

def find_similar_sequences(query_seq, train_seqs_df, train_coords_dict, top_n=5, max_candidates=200):
    """
    OPTIMIZED: Two-stage approach
    1. Fast k-mer filtering to find candidates (100x faster)
    2. Accurate alignment only on top candidates
    This is how production code does it!
    """
    similar_seqs = []
    query_len = len(query_seq)
    
    # STAGE 1: Fast pre-filtering using k-mer similarity (no expensive alignment yet)
    candidates_with_score = []
    for _, row in train_seqs_df.iterrows():
        target_id, train_seq = row['target_id'], row['sequence']
        if target_id not in train_coords_dict: continue
        
        # Quick length filter
        len_diff = abs(len(train_seq) - query_len) / max(len(train_seq), query_len)
        if len_diff > 0.4: continue
        
        # FAST: Use k-mer similarity instead of alignment (100x faster)
        kmer_sim = fast_sequence_similarity(query_seq, train_seq)
        if kmer_sim > 0.1:  # Only keep sequences with some k-mer overlap
            candidates_with_score.append((target_id, train_seq, kmer_sim, train_coords_dict[target_id]))
    
    # Sort by k-mer similarity and take top candidates
    candidates_with_score.sort(key=lambda x: x[2], reverse=True)
    top_candidates = candidates_with_score[:max_candidates]
    
    # STAGE 2: Accurate alignment only on top candidates (much smaller set)
    for target_id, train_seq, kmer_sim, coords in top_candidates:
        # Skip very long sequences - use k-mer score directly
        if len(train_seq) > 2000 or query_len > 2000:
            # Use k-mer similarity as final score for very long sequences
            if kmer_sim > 0.3:
                similar_seqs.append((target_id, train_seq, kmer_sim * 0.8, coords))
            continue
        
        # Only do expensive alignment on promising candidates
        try:
            alignments = list(MODERN_ALIGNER.align(query_seq, train_seq))
            if alignments:
                best_alignment = alignments[0]
                score = best_alignment.score
                max_possible = 2.0 * min(query_len, len(train_seq))
                normalized_score = score / max_possible if max_possible > 0 else 0
                # Combine k-mer and alignment scores for better accuracy
                combined_score = 0.7 * normalized_score + 0.3 * kmer_sim
                similar_seqs.append((target_id, train_seq, combined_score, coords))
        except Exception as e:
            # Fallback: use k-mer score if alignment fails
            if kmer_sim > 0.3:
                similar_seqs.append((target_id, train_seq, kmer_sim * 0.7, coords))
        
        # Early exit if we have enough good matches
        if len(similar_seqs) >= top_n * 2:
            similar_seqs.sort(key=lambda x: x[2], reverse=True)
            return similar_seqs[:top_n]
    
    similar_seqs.sort(key=lambda x: x[2], reverse=True)
    return similar_seqs[:top_n]

def adaptive_rna_constraints(coordinates, sequence, confidence=1.0, iterations=2):
    """
    IMPROVEMENT: Multiple iterations for better refinement
    IMPROVEMENT: Tighter distance constraints (5.8-6.2 instead of 5.5-6.5)
    IMPROVEMENT: Add steric clash prevention
    MEMORY OPTIMIZATION: Replaced distance_matrix with pairwise calculation
    """
    refined_coords = coordinates.copy()
    n_residues = len(sequence)
    
    # IMPROVEMENT: Slightly stronger constraints (0.55 vs 0.5)
    constraint_strength = 0.55 * (1.0 - min(confidence, 0.9))
    
    # IMPROVEMENT: Tighter distance range for more realistic RNA geometry
    seq_min_dist, seq_max_dist = 5.8, 6.2
    target_dist = 6.0
    
    # IMPROVEMENT: Multiple refinement iterations
    for iteration in range(iterations):
        # Sequential distance constraints
        for i in range(n_residues - 1):
            dist = np.linalg.norm(refined_coords[i+1] - refined_coords[i])
            if dist < seq_min_dist or dist > seq_max_dist:
                direction = (refined_coords[i+1] - refined_coords[i]) / (dist + 1e-10)
                adjustment = (target_dist - dist) * constraint_strength
                refined_coords[i+1] = refined_coords[i+1] + direction * adjustment
        
        # IMPROVEMENT: Steric clash prevention (MEMORY OPTIMIZED)
        if iteration == iterations - 1:  # Only on last iteration
            min_allowed_dist = 3.5  # Minimum distance between any two residues
            # MEMORY OPTIMIZATION: Only check nearby residues (within 10 residues) instead of full matrix
            # This reduces memory from O(n²) to O(n) and is more realistic for RNA
            check_window = min(10, n_residues // 2)  # Check up to 10 residues away
            for i in range(n_residues):
                for j in range(i+2, min(i+check_window+1, n_residues)):  # Skip adjacent, limit window
                    dist = np.linalg.norm(refined_coords[j] - refined_coords[i])
                    if dist < min_allowed_dist:
                        direction = refined_coords[j] - refined_coords[i]
                        direction = direction / (np.linalg.norm(direction) + 1e-10)
                        push_distance = (min_allowed_dist - dist) * 0.3
                        refined_coords[i] -= direction * push_distance * 0.5
                        refined_coords[j] += direction * push_distance * 0.5
            
    return refined_coords

def adapt_template_to_query(query_seq, template_seq, template_coords):
    """
    IMPROVEMENT: Better gap filling with smoother interpolation
    IMPROVEMENT: More realistic gap extension
    """
    # Try modern aligner first
    try:
        alignments = list(MODERN_ALIGNER.align(query_seq, template_seq))
        if not alignments:
            return np.zeros((len(query_seq), 3))
        best_alignment = alignments[0]
        # Extract aligned sequences from Alignment object (convert to string)
        aligned_query = str(best_alignment[0])
        aligned_template = str(best_alignment[1])
    except Exception as e:
        # Fallback to pairwise2
        alignments = pairwise2.align.globalms(Seq(query_seq), Seq(template_seq), 2, -1, -3, -0.1, one_alignment_only=True)
        if not alignments:
            return np.zeros((len(query_seq), 3))
        aligned_query, aligned_template = alignments[0].seqA, alignments[0].seqB
    
    new_coords = np.full((len(query_seq), 3), np.nan)
    q_idx, t_idx = 0, 0
    
    for char_q, char_t in zip(aligned_query, aligned_template):
        if char_q != '-' and char_t != '-':
            if t_idx < len(template_coords): 
                new_coords[q_idx] = template_coords[t_idx]
            q_idx += 1
            t_idx += 1
        elif char_q != '-': 
            q_idx += 1
        elif char_t != '-': 
            t_idx += 1

    # IMPROVEMENT: Better gap filling with smoother interpolation
    for i in range(len(new_coords)):
        if np.isnan(new_coords[i, 0]):
            prev_v = next((j for j in range(i-1, -1, -1) if not np.isnan(new_coords[j, 0])), -1)
            next_v = next((j for j in range(i+1, len(new_coords)) if not np.isnan(new_coords[j, 0])), -1)
            if prev_v >= 0 and next_v >= 0:
                # IMPROVEMENT: Smoother interpolation
                w = (i - prev_v) / (next_v - prev_v)
                new_coords[i] = (1-w)*new_coords[prev_v] + w*new_coords[next_v]
            elif prev_v >= 0:
                # IMPROVEMENT: More realistic extension (4.0 instead of 3.0)
                direction = new_coords[prev_v] - (new_coords[prev_v-1] if prev_v > 0 else new_coords[prev_v])
                direction = direction / (np.linalg.norm(direction) + 1e-10) * 4.0
                new_coords[i] = new_coords[prev_v] + direction
            elif next_v >= 0:
                direction = new_coords[next_v+1] - new_coords[next_v] if next_v < len(new_coords)-1 else np.array([4.0, 0, 0])
                direction = direction / (np.linalg.norm(direction) + 1e-10) * 4.0
                new_coords[i] = new_coords[next_v] - direction
            else:
                new_coords[i] = [i*4.0, 0, 0]
    
    return np.nan_to_num(new_coords)

def generate_rna_structure(sequence, seed=None):
    """
    IMPROVEMENT: More realistic RNA structure generation
    IMPROVEMENT: Consider base pairing potential
    """
    if seed: 
        np.random.seed(seed)
        random.seed(seed)
    
    n = len(sequence)
    coords = np.zeros((n, 3))
    
    # IMPROVEMENT: More realistic initial structure with slight curvature
    base_pairing = {'G': 'C', 'C': 'G', 'A': 'U', 'U': 'A'}
    
    for i in range(1, n):
        # IMPROVEMENT: More consistent step size (4.0-4.5 instead of 3.0-5.0)
        step_size = random.uniform(4.0, 4.5)
        
        # IMPROVEMENT: Add slight helical twist
        angle = i * 0.1  # Small rotation per residue
        rotation = R.from_euler('z', angle, degrees=False)
        step_vector = np.array([step_size, 0, 0])
        step_vector = rotation.apply(step_vector)
        
        coords[i] = coords[i-1] + step_vector
    
    return coords

# === 3. PREDICT - IMPROVED ===

def predict_rna_structures(sequence, target_id, train_seqs_df, train_coords_dict, n_predictions=5):
    """
    IMPROVEMENT: Reduced noise (0.15 instead of 0.25) for more stable predictions
    IMPROVEMENT: Better handling of low-confidence templates
    """
    predictions = []
    similar_seqs = find_similar_sequences(sequence, train_seqs_df, train_coords_dict, top_n=n_predictions)
    
    if similar_seqs:
        for i, (template_id, template_seq, similarity, template_coords) in enumerate(similar_seqs):
            adapted = adapt_template_to_query(sequence, template_seq, template_coords)
            
            # IMPROVEMENT: Multiple refinement iterations
            refined = adaptive_rna_constraints(adapted, sequence, confidence=similarity, iterations=2)
            
            # IMPROVEMENT: Reduced noise (0.15 vs 0.25) for more stable predictions
            # Less noise for high-confidence templates
            random_scale = max(0.03, (0.5 - similarity) * 0.15)
            refined += np.random.normal(0, random_scale, refined.shape)
            
            predictions.append(refined)
                
    # IMPROVEMENT: Better fallback structure generation
    while len(predictions) < n_predictions:
        seed_value = hash(target_id) % 10000 + len(predictions) * 1000
        de_novo = generate_rna_structure(sequence, seed=seed_value)
        # Apply constraints to de novo structures too
        de_novo_refined = adaptive_rna_constraints(de_novo, sequence, confidence=0.3, iterations=1)
        predictions.append(de_novo_refined)
    
    return predictions[:n_predictions]

# === 4. LOOP & SAVE (MEMORY OPTIMIZED) ===
# MEMORY OPTIMIZATION: Write incrementally instead of accumulating in memory
import gc
import os
import sys

# Memory monitoring function
def get_memory_usage():
    """Get current memory usage in MB"""
    try:
        import psutil
        process = psutil.Process(os.getpid())
        mem_info = process.memory_info()
        return mem_info.rss / 1024 / 1024  # Convert to MB
    except ImportError:
        # Fallback: try resource module (Unix)
        try:
            import resource
            mem_info = resource.getrusage(resource.RUSAGE_SELF)
            return mem_info.ru_maxrss / 1024  # KB to MB (Linux) or already MB (macOS)
        except:
            return None

def get_total_memory():
    """Get total system memory in MB"""
    try:
        import psutil
        return psutil.virtual_memory().total / 1024 / 1024
    except:
        return None

def check_memory_warning(threshold_mb=None, warning_mb=None):
    """
    Check memory usage and warn if high.
    Defaults: warning at 70% of system memory, critical at 85%
    For Kaggle 29GB RAM: warning at ~20GB, critical at ~25GB
    """
    mem_usage = get_memory_usage()
    if mem_usage is None:
        return None
    
    # Auto-detect thresholds based on system memory
    total_mem = get_total_memory()
    if threshold_mb is None:
        if total_mem:
            threshold_mb = total_mem * 0.85  # 85% of total memory
        else:
            threshold_mb = 25000  # Default for 29GB Kaggle (85% of 29GB)
    
    if warning_mb is None:
        if total_mem:
            warning_mb = total_mem * 0.70  # 70% of total memory
        else:
            warning_mb = 20000  # Default for 29GB Kaggle (70% of 29GB)
    
    if mem_usage > threshold_mb:
        print(f"CRITICAL: Memory usage is {mem_usage:.1f} MB / {total_mem:.1f} MB ({mem_usage/total_mem*100:.1f}%)" if total_mem else f"CRITICAL: Memory usage is {mem_usage:.1f} MB (>{threshold_mb:.0f} MB threshold)")
        print("   Consider reducing batch size or sequence length!")
        gc.collect()  # Force cleanup
    elif mem_usage > warning_mb:
        print(f"WARNING: Memory usage is {mem_usage:.1f} MB / {total_mem:.1f} MB ({mem_usage/total_mem*100:.1f}%)" if total_mem else f"WARNING: Memory usage is {mem_usage:.1f} MB (>{warning_mb:.0f} MB)")
    
    return mem_usage

# Prepare output file
cols = ['ID', 'resname', 'resid'] + [f'{c}_{i}' for i in range(1,6) for c in ['x','y','z']]
first_write = True

print(f"Starting processing of {len(test_seqs)} sequences...")
total_system_mem = get_total_memory()
initial_mem = get_memory_usage()
if initial_mem:
    print(f"Initial memory usage: {initial_mem:.1f} MB", end="")
    if total_system_mem:
        print(f" / {total_system_mem:.1f} MB total ({initial_mem/total_system_mem*100:.1f}%)")
    else:
        print()

import time

for idx, row in test_seqs.iterrows():
    target_id, sequence = row['target_id'], row['sequence']
    seq_start_time = time.time()
    
    if idx % 5 == 0: 
        print(f"Processing {idx+1}/{len(test_seqs)} - Sequence length: {len(sequence)}")
        # Check memory and warn if high
        mem_usage = check_memory_warning()
        if mem_usage:
            print(f"   Current memory: {mem_usage:.1f} MB")
        # Force garbage collection periodically
        gc.collect()
    
    # PERFORMANCE OPTIMIZATION: Skip very long sequences or use fallback
    if len(sequence) > 2000:
        print(f"   WARNING: Very long sequence ({len(sequence)}), using simplified prediction")
    
    try:
        preds = predict_rna_structures(sequence, target_id, combined_seqs, combined_coords_dict)
        
        # Build rows for this sequence
        batch_rows = []
        for j in range(len(sequence)):
            res = {'ID': f"{target_id}_{j+1}", 'resname': sequence[j], 'resid': j+1}
            for i in range(5):
                res[f'x_{i+1}'], res[f'y_{i+1}'], res[f'z_{i+1}'] = preds[i][j]
            batch_rows.append(res)
        
        # Write incrementally
        batch_df = pd.DataFrame(batch_rows)
        batch_df[cols].to_csv('submission.csv', mode='a' if not first_write else 'w', 
                              header=first_write, index=False)
        first_write = False
        
        seq_time = time.time() - seq_start_time
        if seq_time > 60:  # Warn if sequence takes more than 1 minute
            print(f"   Sequence {idx+1} took {seq_time:.1f}s to process")
        
        # Clean up memory immediately after each sequence
        del preds, batch_rows, batch_df
        gc.collect()
        
    except Exception as e:
        print(f"   ERROR processing sequence {idx+1} ({target_id}): {str(e)}")
        print(f"   Using fallback structure generation")
        # Fallback: generate simple structure
        try:
            fallback_preds = []
            for i in range(5):
                seed_value = hash(target_id) % 10000 + i * 1000
                de_novo = generate_rna_structure(sequence, seed=seed_value)
                de_novo_refined = adaptive_rna_constraints(de_novo, sequence, confidence=0.3, iterations=1)
                fallback_preds.append(de_novo_refined)
            
            batch_rows = []
            for j in range(len(sequence)):
                res = {'ID': f"{target_id}_{j+1}", 'resname': sequence[j], 'resid': j+1}
                for i in range(5):
                    res[f'x_{i+1}'], res[f'y_{i+1}'], res[f'z_{i+1}'] = fallback_preds[i][j]
                batch_rows.append(res)
            
            batch_df = pd.DataFrame(batch_rows)
            batch_df[cols].to_csv('submission.csv', mode='a' if not first_write else 'w', 
                                  header=first_write, index=False)
            first_write = False
            
            del fallback_preds, batch_rows, batch_df
            gc.collect()
        except Exception as e2:
            print(f"   CRITICAL: Fallback also failed: {str(e2)}")
    
    # More frequent GC for memory-intensive operations
    if idx % 5 == 0:
        gc.collect()
        # Check memory after cleanup
        mem_usage = check_memory_warning()
        if mem_usage:
            print(f"   Memory after cleanup: {mem_usage:.1f} MB")

final_mem = get_memory_usage()
if final_mem and initial_mem:
    print(f"\nProcessing complete!")
    print(f"Final memory usage: {final_mem:.1f} MB")
    print(f"Memory increase: {final_mem - initial_mem:.1f} MB")
print("Submission.csv generated!")

0.365 submission score
